This notebook debugs `hits`

20250720

In [ ]:
import straxion
import matplotlib.pyplot as plt
import numpy as np

# Apply the style
plt.style.use('../../xenonnt.mplstyle')

st = straxion.qualiphide()
st.set_config(
    dict(
        daq_input_dir="timeS429", 
        iq_finescan_dir="finescan_iq428"
    )
)

st.is_stored(
    "timeS429", # run id
    "records", # data type
)

In [ ]:
#hits = st.get_array("timeS429", "hits")
hits = st.get_array("timeS429", "hits")
hits = hits[hits["channel"] == 0]

In [ ]:
st.data_info("hits")

In [ ]:
plt.hist(hits["width"])
plt.xlabel("Width [20us]")
plt.ylabel("Count")
plt.show()

In [ ]:
plt.hist(hits["amplitude_ma_max"], alpha=0.5, label="60 samples since start")
plt.hist(hits["amplitude_ma_max_ext"], alpha=0.5, label="100 samples since start")
plt.xlabel("Maximum Amplitude [rad]")
plt.ylabel("Count")
plt.yscale("log")
plt.legend()
plt.show()

In [ ]:
hit = hits[30]
expected_endtime = hit["time"] + hit["length"] * hit["dt"]
expected_endtime - hit["endtime"]

In [ ]:
for i in range(len(hits)):
    plt.plot(hits[i]["data_theta"], label="Flipped and Shifted",color="black", alpha=0.5, lw=1)
    plt.plot(hits[i]["data_theta_moving_average"], label="Moving Averaged", color="tab:blue", lw=2)
    plt.plot(hits[i]["data_theta_convolved"], label="Convolved with Pulse Kernel", color="tab:orange", lw=2)
    plt.axhline(hits[i]["hit_threshold"], color="tab:red", ls=":")
    plt.axhline(0, color="black", ls=":")
    plt.axhline(hits[i]["amplitude_ma_max"], color="tab:blue", ls="--")
    plt.axhline(hits[i]["amplitude_convolved_max"], color="tab:orange", ls="--")
    plt.axhline(hits[i]["amplitude_convolved_max_ext"], color="tab:orange", ls=":")
    plt.axhline(hits[i]["amplitude_ma_max_ext"], color="tab:blue", ls=":")
    plt.title("Hit #"+str(i))
    plt.xlabel("Samples [20us]")
    plt.legend()
    plt.show()

In [ ]:
r = st.get_array(
    "timeS429", # run id
    "records", # data type
)[0] # Only read the first record, as the rest are the same.

In [ ]:
slice_sample_length = 20_000
total_sample_length = r["length"]

convolved_std = np.std(r['data_theta_convolved'])
hit_id = 0

#for i in range(5):
for i in range(int(total_sample_length / slice_sample_length)):
    sample_range = (i * slice_sample_length, (i + 1) * slice_sample_length)

    times_start = r["time"]
    times_us = (
        r['time'] + np.arange(r['length']) * r['dt'] - times_start
    ) / 1e3
    times_us = times_us[sample_range[0]:sample_range[1]]

    plt.figure(figsize=(10, 5))
    plt.plot(
        times_us, 
        r['data_theta'][sample_range[0]:sample_range[1]], 
        label="Flipped and Shifted", 
        color="black",
        alpha=0.5,
        lw=1
    )
    plt.plot(
        times_us, 
        r['data_theta_moving_average'][sample_range[0]:sample_range[1]], 
        label="Moving Averaged", 
        color="tab:blue",
        lw=2
    )
    plt.plot(
        times_us, 
        r['data_theta_convolved'][sample_range[0]:sample_range[1]], 
        label="Convolved with Pulse Kernel", 
        color="tab:orange",
        lw=2
    )

    # Add hit time ranges
    slice_hits = hits[
        (hits['time'] >= times_start + sample_range[0] * r['dt']) &
        (hits['time'] < times_start + sample_range[1] * r['dt'])
    ]
    for hit in slice_hits:
        hit_start_us = (hit['time'] - times_start) / 1e3
        hit_end_us = (hit['endtime'] - times_start) / 1e3
        plt.axvspan(hit_start_us, hit_end_us, alpha=0.2, color='red')
        plt.axvline(hit["aligned_at_records_i"] * r['dt'] / 1e3, color="tab:red")
        plt.text(hit_start_us, hit["amplitude_ma_max"], "#"+str(hit_id), color="tab:red")
        hit_id += 1

    # Show threshold and baseline
    
    plt.axhline(hits[i]["hit_threshold"], color="tab:red", ls=":")
    plt.axhspan(-convolved_std, convolved_std, color="tab:orange", alpha=0.2, zorder=0)

    plt.xlim(times_us[0], times_us[-1])

    plt.legend()
    plt.xlabel("Time Since Run Start [us]")
    plt.ylabel("Theta [rad]")
    plt.title(f"Slice {i+1} of {int(total_sample_length / slice_sample_length)}")
    plt.show()